# LAB: Feature Selection

En este lab vamos a explorar selección de características. Seguiremos trabajando sobre el dataset de Titanic.

Ante que nada, carguemos algunas cosas:

- Paquetes estándar
- El dataset

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('train.csv')

In [ ]:
X = df[[u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare', u'Embarked']]
y = df[u'Survived']

## 1. Preprocesamiento

Va a ser necesario realizar algunas tareas de preprocesamiento para este dataset:

1. Estandarizar la edad (`age`) y la tarifa (`fare`)
2. Construir algunas variables dummies para algunas variables relevantes (tales como `P_class`, `Embarked`, `Male`)

In [ ]:
df.head()

In [ ]:
cols = df[['Pclass','Sex', 'Embarked']]
dummies = pd.get_dummies(cols, drop_first=True)
dummies.head()

In [ ]:
cols2 = df[['Age','Fare']].fillna(df[['Age','Fare']].mean())

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
std = scaler.fit_transform(cols2)
std = pd.DataFrame(std, columns=['std_Age','std_Fare'])

In [ ]:
#dummies.reset_index(drop=False)
#std.reset_index(drop=False)

In [ ]:
X = pd.concat([dummies, std, df[['SibSp','Parch']]],axis=1)

## 2. Feature selection

Utilicemos el método "SelectKBest" de scikit learn a ver cuáles son las top 5 características.

Cuáles son?


Guardémoslas en una variable llamada "kbest_columns"

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
selector = SelectKBest(f_classif, k=5)
selected_data = selector.fit_transform(X, y)
kbest_columns = X.columns[selector.get_support()]
Xbest = pd.DataFrame(selected_data, columns=kbest_columns)
Xbest.head()

## 3. Eliminación recursiva de características

En Scikit Learn también vamos a encontrar una clase para realizar una eliminación recursiva de características. La misma se llama "RFECV". Usémosla en combinación de un modelo de regresión logística para ver qué características serán conservadas con este método.

Guardémoslas en una variable llamada "rfecv_columns"

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [ ]:
estimator = LogisticRegression()
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)
rfecv_columns = X.columns[selector.support_]
rfecv_columns

## 4. Coeficientes de Regresión Logística

Veamos si los coeficientes de una RL se condicen.

- Creá un modelo de regresión logística
- Ejecutá un grid search sobre los parámetros "penalty type" y "C strength" para encontrar la mejor combinación
- Ordená los coeficientes obtenidos por valor absoluto (módulo). El top 5 coincide con los de arriba? Por qué/Por qué no? (Pista: Están todos los valores en la misma escala?)

Guardemos las que querramos mantener en una variable llamada "lr_columns"

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = GridSearchCV(LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                                            'penalty': ['l1', 'l2']})
model.fit(X, y)

In [ ]:
model.best_estimator_

In [ ]:
model.best_score_

In [ ]:
coeffs = pd.DataFrame(model.best_estimator_.coef_, columns = X.columns)
coeffs_t = coeffs.transpose()
coeffs_t.columns = ['Surv coeff']
coeffs_t.abs().sort_values('Surv coeff', ascending=False)

In [ ]:
lr_columns = coeffs.columns[(coeffs.abs() > 0.3).values[0]]
lr_columns

## 5. Comparar sets de características

Usá el "best estimator" del punto anterior sobre los sets de características obtenidos:

- "kbest_columns"
- "rfecv_columns"
- "lr_columns"
- "all_columns"

Usá validación cruzada (cross_val_score) para evaluar los modelos 
Preguntas:

- Cuál obtuvo mejores resultados?
- Hay diferencias signigicativas?
- Cuál es la mejor opción? Por qué?

In [ ]:
from sklearn.cross_validation import cross_val_score

def score(X):
    scores = cross_val_score(model.best_estimator_, X, y)
    return scores.mean(), scores.std()

all_scores = [
    score(X[kbest_columns]),
    score(X[rfecv_columns]),
    score(X[lr_columns]),
    score(X)]

pd.DataFrame(all_scores, columns=['mean score', 'std score'], index = ['kbest', 'rfecv', 'lr', 'all'])


## Bonus

Creá un grágico de barras para mostrar los coeficientes de la regresión logística.

In [ ]:
coeffs_t.sort_values('Surv coeff').plot(kind='bar')